In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input

In [2]:
df = pd.read_csv('train.csv').drop(columns=['Id'])
X = np.array(df.iloc[:, 1:])
y = to_categorical(np.array(df.iloc[:, 0]))

# Convert the training and test images into 3 channels
X = np.dstack([X] * 3)
# Reshape images as per the tensor format required by tensorflow
X = X.reshape(-1, 28,28,3)
X = np.asarray([img_to_array(array_to_img(im, scale=False).resize((150,150))) for im in X])
X = X.astype('float32')/255

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size = 0.5, random_state = 42)

In [4]:
# Create the base model of VGG19
vgg19 = VGG19(weights='imagenet', include_top=False, input_shape = (150, 150, 3), classes = 5)
vgg19.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [5]:
# Preprocessing the input 
X_train = preprocess_input(X_train)
X_val = preprocess_input(X_val)
X_test = preprocess_input(X_test)

In [6]:
# Extracting features
train_features = vgg19.predict(np.array(X_train), batch_size=256)
test_features = vgg19.predict(np.array(X_test), batch_size=256)
val_features = vgg19.predict(np.array(X_val), batch_size=256)
# Current shape of features
print(train_features.shape, "\n",  test_features.shape, "\n", val_features.shape)

In [8]:
# Flatten extracted features
train_features = np.reshape(train_features, (48000, 4*4*512))
test_features = np.reshape(test_features, (6000, 4*4*512))
val_features = np.reshape(val_features, (6000, 4*4*512))

In [9]:
model = Sequential([
    Dense(512, activation='relu', input_dim=4*4*512),
    Dropout(0.5),
    Dense(5, activation="softmax")
])
model.compile(optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999), loss="categorical_crossentropy", metrics=["accuracy"])

In [10]:
# Train the the model
history = model.fit(train_features, y_train, batch_size=128, epochs=50, verbose=1, validation_data=(val_features, y_val))

Train on 48000 samples, validate on 6000 samples
Epoch 1/50
48000/48000 [==============================] - 3s 59us/sample - loss: 1.5588 - accuracy: 0.3438 - val_loss: 1.2661 - val_accuracy: 0.4767
Epoch 2/50
48000/48000 [==============================] - 2s 43us/sample - loss: 1.2444 - accuracy: 0.4401 - val_loss: 1.1444 - val_accuracy: 0.5095
Epoch 3/50
48000/48000 [==============================] - 2s 43us/sample - loss: 1.1830 - accuracy: 0.4715 - val_loss: 1.1033 - val_accuracy: 0.5433
Epoch 4/50
48000/48000 [==============================] - 2s 42us/sample - loss: 1.1382 - accuracy: 0.4927 - val_loss: 1.0085 - val_accuracy: 0.5652
Epoch 5/50
48000/48000 [==============================] - 2s 42us/sample - loss: 1.1196 - accuracy: 0.5033 - val_loss: 0.9708 - val_accuracy: 0.5887
Epoch 6/50
48000/48000 [==============================] - 2s 42us/sample - loss: 1.1093 - accuracy: 0.5087 - val_loss: 1.0231 - val_accuracy: 0.5737
Epoch 7/50
48000/48000 [==============================] -

In [11]:
score = model.evaluate(test_features, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.8325433111190796
Test accuracy: 0.65533334


In [12]:
# plot the loss and accuracy
import matplotlib.pyplot as plt
%matplotlib inline

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

plt.title('Training and validation accuracy')
plt.plot(epochs, acc, 'red', label='Training acc')
plt.plot(epochs, val_acc, 'blue', label='Validation acc')
plt.legend()

plt.figure()
plt.title('Training and validation loss')
plt.plot(epochs, loss, 'red', label='Training loss')
plt.plot(epochs, val_loss, 'blue', label='Validation loss')

plt.legend()

plt.show()

KeyError: 'acc'

Although the lines of loss and accuracy for both training and validation follow similar trends, there're some space between their values.

## Classification Report
I can summarize the performance of my classifier as follows:

In [19]:
# get the predictions for the test data
predicted_classes = model.predict_classes(test_features)

# get the indices to be plotted
y_true = data_test.iloc[:, 0]
correct = np.nonzero(predicted_classes==y_true)[0]
incorrect = np.nonzero(predicted_classes!=y_true)[0]

In [20]:
from sklearn.metrics import classification_report
target_names = ["Class {}".format(i) for i in range(10)]
print(classification_report(y_true, predicted_classes, target_names=target_names))

             precision    recall  f1-score   support

    Class 0       0.63      0.83      0.71      1000
    Class 1       0.94      0.95      0.95      1000
    Class 2       0.68      0.56      0.61      1000
    Class 3       0.81      0.79      0.80      1000
    Class 4       0.62      0.76      0.68      1000
    Class 5       0.77      0.95      0.85      1000
    Class 6       0.50      0.27      0.35      1000
    Class 7       0.89      0.77      0.82      1000
    Class 8       0.87      0.94      0.90      1000
    Class 9       0.91      0.85      0.88      1000

avg / total       0.76      0.77      0.76     10000



The model underperforms for Class 2 and 6. It lacks precision for class 0 and 4, additionally. 

In [21]:
test_features[correct].shape

(7664, 8192)